### Load Francy Package

In [3]:
LoadPackage("francy");
LoadPackage("digraph");
LoadPackage("num");

#I  Please load package NormalizInterface or 4ti2Interface
#I  to have extended functionalities.
#I  Please load package SingularInterface or singular (not both)
#I  or GradedModules to have extended functionalities.


true

true

true

### An attempt to LatticeSubgroups By Markus

In [8]:
DisplayLatticeSubgroups := function(L)
    local cls, len, sz, max, rep, z, t, i, j, k, graphHasse, nodes, last;
    cls:=ConjugacyClassesSubgroups(L);
    len:=[];
    sz:=[];
    for i in cls do
        Add(len,Size(i));
        AddSet(sz,Size(Representative(i)));
    od;

    graphHasse := Graph(GraphType.UNDIRECTED);
#SetShowNeighbours(graphHasse,true);

    nodes := [];
    sz:=Reversed(sz);
    # subgroup nodes, also acccording to size
    for i in [1..Length(cls)] do
        nodes[i] := [];
        for j in [1..len[i]] do
            if len[i]=1 then
                nodes[i][j] := Shape(ShapeType.DIAMOND, String(i));
                SetLayer(nodes[i][j], -Size(Representative(cls[i])));
                # AppendTo(file,"\"",i,"x",j,"\" [label=\"",i,"\", shape=box];\n");
            else
                nodes[i][j] := Shape(ShapeType.CIRCLE, String(i));
                SetLayer(nodes[i][j], -Size(Representative(cls[i])));
                # AppendTo(file,"\"",i,"x",j,"\" [label=\"",i,"-",j,"\", shape=circle];\n");
            fi;
            Add(graphHasse, nodes[i][j]);
        od;
    od;

    last:=rec(o:=0,n:=0);
    for i in [1..Length(cls)] do
        for j in [1..len[i]] do
            if Layer(nodes[i][j]) <> last.o then
                last.o := Layer(nodes[i][j]);
                last.n := last.n - 2;
            fi;
            SetLayer(nodes[i][j], last.n);
        od;
    od;

    max:=MaximalSubgroupsLattice(L);
    for i in [1..Length(cls)] do
        for j in max[i] do
            rep:=ClassElementLattice(cls[i],1);
            for k in [1..len[i]] do
                if k=1 then
                    z:=j[2];
                else
                    t:=cls[i]!.normalizerTransversal[k];
                    z:=ClassElementLattice(cls[j[1]],1); # force computation of transv.
                    z:=cls[j[1]]!.normalizerTransversal[j[2]]*t;
                    z:=PositionCanonical(cls[j[1]]!.normalizerTransversal,z);
                fi;
                Add(graphHasse, Link(nodes[i][k],nodes[j[1]][z]));
          od;
        od;
      od;
    return graphHasse;
end;;
L:=LatticeSubgroups(DihedralGroup(44));;
canvas := Canvas("Lattice Subgroups");;
Add(canvas, DisplayLatticeSubgroups(L));;
Draw(canvas);

## Digraphs

In [36]:
G := DihedralGroup(8);; #SymmetricGroup(5);; #FreeGroup( "a", "b" );;
as := AllSubgroups(G);;
d := Digraph(as, {H, K} -> IsSubgroup(H, K));;

vertices := DigraphVertices(d);;
edges := DigraphEdges(d);;

graph := Graph(GraphType.DIRECTED);;
#SetDrag(graph, true);;
nodes := [];;

simpleGroupMessage := FrancyMessage(FrancyMessageType.INFO, "Simple Groups", "A group is simple if it is nontrivial and has no nontrivial normal subgroups.");;

IsGroupSimple := function(i)
    Add(canvas, simpleGroupMessage);;
    if IsSimpleGroup(as[i]) then
        Add(canvas, FrancyMessage("Simple", Concatenation("The vertex ", String(i), ", representing the subgroup ", String(as[i]), ", is simple.")));;
    else
        Add(canvas, FrancyMessage("Not Simple", Concatenation("The vertex ", String(i), ", representing the subgroup ", String(as[i]), ", is not simple.")));;
    fi;;
    return Draw(canvas);
end;;
    
for i in vertices do;
    nodes[i] := Shape(ShapeType.CIRCLE, String(i));;
#SetColor(nodes[i], "#0c9460");;
    Add(nodes[i], Menu("Is this subgroup simple?", Callback(IsGroupSimple, [i])));;
    Add(graph, nodes[i]);;
od;;

for i in edges do
    Add(graph, Link(nodes[i[1]], nodes[i[2]]));;
od;;

canvas := Canvas(Concatenation("Subgroups Digraph of ", String(G)));;
Add(canvas, graph);;
Draw(canvas);